In [1]:
import os

from dotenv import load_dotenv

load_dotenv()

# if not os.getenv("OPENAI_API_KEY"):
#     raise ValueError("OPENAI_API_KEY is not set in .env file")
if not os.getenv("ANTHROPIC_API_KEY"):
    raise ValueError("ANTHROPIC_API_KEY is not set in .env file")
if not os.getenv("GOOGLE_API_KEY"):
    raise ValueError("GOOGLE_API_KEY is not set in .env file")
if not os.getenv("DEEPSEEK_API_KEY"):
    raise ValueError("DEEPSEEK_API_KEY is not set in .env file")

In [2]:
import sys
import json
from datetime import datetime
import pandas as pd

# LangChain関連
from langchain_anthropic import ChatAnthropic
from langchain_deepseek import ChatDeepSeek
from langchain_google_genai import ChatGoogleGenerativeAI

# OpenAI Evals関連

# Add the project root directory to Python path
project_root = os.path.abspath(os.path.join(os.path.dirname("__file__"), ".."))
sys.path.append(project_root)

/Users/hiromu.nakamura.001/ghq/github.com/po3rin/llmmj/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configure logging to see error messages
import logging

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

os.environ["GOOGLE_ADK_LOG_LEVEL"] = "WARNING"

In [4]:
# モデル設定（APIキーが必要）
try:
    # # OpenAI GPT-4
    # gpt4 = ChatOpenAI(model_name="gpt-4o", temperature=0)

    # Claude (Anthropic APIキーが必要)
    claude_sonnet = ChatAnthropic(model="claude-sonnet-4-20250514", temperature=0)
    # claude_opus = ChatAnthropic(model="claude-opus-4-20250514", temperature=0)
    claude_3_7_sonnet = ChatAnthropic(model="claude-3-7-sonnet-20250219", temperature=0)

    # Google Gemini
    gemini = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash-preview-05-20", temperature=0
    )

    # DeepSeek
    deepseek = ChatDeepSeek(model="deepseek-chat", temperature=0)

except Exception as e:
    print(f"モデル初期化エラー: {e}")
    raise e

In [5]:
# from generator.generator import MahjongQuestionGenerator
# from prompts.prompts import generate_question_prompt_template

# generator = MahjongQuestionGenerator(
#     model=claude_3_7_sonnet, query_template=generate_question_prompt_template
# )
# result = generator.generate_question(query="Please give me a mahjong scoring problem. Give me a problem where the answer is 3 han and 50 fu.")
# print(result)

In [6]:
# from generator.generator import MahjongQuestionGenerator
# from prompts.prompts import generate_question_with_cot_and_rule_prompt_template

# generator = MahjongQuestionGenerator(
#     model=gemini, query_template=generate_question_with_cot_and_rule_prompt_template
# )
# result = generator.generate_question(query="Please give me a mahjong scoring problem. Give me a problem where the answer is 3 han and 50 fu.")
# print(result)

In [7]:
# # 評価システム初期化(with tools)
# from generator.generator import MahjongQuestionGenerator
# from prompts.prompts import generate_question_with_tools_prompt_template

# generator = MahjongQuestionGenerator(
#     model=gemini,
#     use_tools=True,
#     query_template=generate_question_with_tools_prompt_template,
# )

# # Use the async version in Jupyter notebook
# result = generator.generate_question(query="Please give me a mahjong scoring problem. Give me a problem where the answer is 3 han and 50 fu.")
# print(result)

In [8]:
# e = MultiModelEvaluator(models=[claude_sonnet], query_template=generate_question_prompt_template)
# df = e.evals(dataset)
# df

In [9]:
with open("../dataset/queries.json", "r") as f:
    dataset = json.load(f)

In [10]:
# with open("../dataset/queries.min.json", "r") as f:
#     dataset = json.load(f)

In [11]:
# dataset

In [12]:
models = {"claude_sonnet": claude_sonnet, "gemini": gemini, "deepseek": deepseek}

In [13]:
from datetime import datetime

from evaluator.evaluator import MultiModelEvaluator
from prompts.prompts import generate_question_prompt_template


for k, m in models.items():
    e = MultiModelEvaluator(
        models=[m], query_template=generate_question_prompt_template
    )
    df = e.evals(dataset)

    df.to_csv(
        f"../dist/zeroshot/evals-gen-question-raw-prompt-{k}-{datetime.now().strftime('%Y%m%d')}.csv",
        index=False,
    )
    print(f"{k} done :)")

In [15]:
from prompts.prompts import generate_question_with_cot_and_rule_prompt_template

for k, v in models.items():
    e = MultiModelEvaluator(
        models=[v], query_template=generate_question_with_cot_and_rule_prompt_template
    )
    df = e.evals(dataset)

    df.to_csv(
        f"../dist/cot_and_rule/evals-gen-question-with-cot-and-rule-prompt-{k}-{datetime.now().strftime('%Y%m%d')}.csv",
        index=False,
    )

In [17]:
from prompts.prompts import generate_question_with_tools_prompt_template

for k, v in models.items():
    e = MultiModelEvaluator(
        models=[v],
        query_template=generate_question_with_tools_prompt_template,
        use_tools=True,
    )
    df = e.evals(dataset)

    df.to_csv(
        f"../dist/tools/evals-gen-question-with-tools-prompt-{k}-{datetime.now().strftime('%Y%m%d')}.csv",
        index=False,
    )

In [ ]:
from evaluator.agents_evaluator import MahjongMultiAgentsEvaluator

evaluator = MahjongMultiAgentsEvaluator(runner_type="sequential")

df = evaluator.evals(dataset)

df.to_csv(
    f"../dist/sequencial_multi_agents/evals-gen-question-with-sequential-agent-{datetime.now().strftime('%Y%m%d')}.csv",
    index=False,
)

In [18]:
from evaluator.agents_evaluator import MahjongMultiAgentsEvaluator

evaluator = MahjongMultiAgentsEvaluator(runner_type="loop")

df = evaluator.evals(dataset)

INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/publishers\/google\/models\/gemini.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'

✅ Agent 'mahjong_score_question_generator_agent' redefined.
✅ Agent 'mahjong_score_question_checker_agent' redefined.
✅ Root Agent 'mahjong_score_question_generation_supervisor_agent' created using stateful tool and output_key.
✅ Agent 'final_output_json_generator_agent' redefined.
✅ New InMemorySessionService created for state demonstration.


INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
麻雀のスコア計算問題を作成します。

---

**問題:**

あなたは南家（プレイヤーの座風: 南）で、場風は東です。
ドラ表示牌は `7z` (赤ドラ) です。

以下の手牌でリーチを宣言し、`4s` をツモアガリしました。この時の点数を計算してください。

**手牌 (アガリ形):**
`1m 2m 3m` `4p 5p 6p` `7s 8s 9s` `7m 7m` `2s 3s 4s` (最後の `4s` はツモアガリした牌)

**状況:**
*   あなた: 南家
*   場風: 東
*   ドラ表示牌: `7z` (赤ドラ)
*   アガリ方: リーチ後のツモアガリ

---

**思考プロセス:**

1.  **アガリ形と待ちの確認:**
    *   手牌は `1m2m3m`, `4p5p6p`, `7s8s9s`, `7m7m`, `2s3s4s` の4つの順子と1つの対子で構成されている。
    *   `4s` をツモアガリして `2s3s` から `2s3s4s` の順子を完成させている。これはリャンメン待ち（2s3sで1sまたは4s待ち）でのアガリ。
    *   すべての面子（メンツ）が順子である。

2.  **役の判定 (Han 計算):**
    *   **リーチ:** リーチを宣言しているので、**1ハン**。
    *   **ツモ:** ツモアガリなので、**1ハン**。
    *   **ピンフ:**
        *   すべての面子が順子である。
        *   アタマ（雀頭）が `7m` であり、場風（東）でも自風（南）でもなく、三元牌でもないので、役牌ではない。
        *   リャンメン待ち（`2s3s` で `4s` をツモアガリ）でアガっている。
        *   手牌は鳴いていない（リーチしているため）。
        *   これらすべてがピンフの条件を満たすため、**1ハン**。
    *   **ドラ:** 

✅ New InMemorySessionService created for state demonstration.


INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
## 麻雀得点計算問題

以下の状況における、ロン上がりの点数を計算してください。

### 状況
*   **プレイヤー**: 子 (非親)
*   **場風 (Round Wind)**: 東 (1z)
*   **自風 (Seat Wind)**: 南 (2z)
*   **ドラ表示牌 (Dora Indicator Tile)**: 6s (六索)
*   **手牌 (Hand in Tenpai, after Riichi declaration)**:
    7z 7z 7z (閉), 5m 5m 5m (閉), 1p 2p 3p (面), 4p 4p (雀頭), 5s 6s (聴牌形)
*   **上がり方**: 7s (七索) でロン上がり。リーチを宣言済み。

### あなたが計算すべき項目
1.  役 (Yaku) の種類と翻数 (Han)
2.  符 (Fu) の計算と最終的な符
3.  最終的な点数 (非親ロン)

---

### Chain of Thought

1.  **役 (Yaku) の計算**:
    *   手牌は全て面子手で完成している。
    *   **リーチ**: リーチを宣言しているので、1翻。
    *   **役牌 (7z)**: 紅中 (7z) の暗刻 (閉鎖された刻子) があるため、役牌として1翻。
    *   **ドラ (7s)**: ドラ表示牌が6sなので、ドラは7s。上がり牌が7sであり、手牌中に7sが1枚あるため、ドラ1枚につき1翻として、ドラ1。
    *   **その他**: 門前清一色ではないため、ピンフはつかない（刻子があるため）。タンヤオではない（1pがあるため）。イーペーコー、三色同順、一気通貫など他の役はない。
    *   **合計翻数**: リーチ(1) + 役牌(1) + ドラ(1) = 3翻

2.  **符 (Fu) の計算**:
    *   **基本符**: 面前でのロン上がりなので、基本符は20符。
  

✅ New InMemorySessionService created for state demonstration.


INFO:google_adk.google.adk.models.google_llm:
LLM Response:
-----------------------------------------------------------
Text:
### 麻雀スコア計算問題

**状況設定:**
*   **プレイヤー:** 南家 (非親)
*   **場風:** 北 (4z)
*   **ドラ表示牌:** 3索 (3s)

**プレイヤーの和了手牌 (リーチを宣言し、待ちの状態で表示。自摸牌/ロン牌は含まず):**
1z1z1z 7z7z7z 2m3m4m 3s4s5s 2p

**和了状況:**
プレイヤーは上記の手牌で**リーチを宣言**し、**2p**をロンして和了しました。
この手牌は、東の暗槓 (1z1z1z1z) と紅中の暗刻 (7z7z7z) を含み、2pが暗刻となることで和了形が完成します。（つまり、手牌の構成は東の暗槓、紅中の暗刻、2m3m4mの順子、3s4s5sの順子、そして2p2pの対子で完成となります）。

**質問:**
この和了の飜数と符数を計算し、最終的な点数を導き出してください。

---

**思考プロセス:**

1.  **手牌の分解と確認:**
    *   暗槓: 1z1z1z1z (東)
    *   暗刻: 7z7z7z (紅中)
    *   順子: 2m3m4m
    *   順子: 3s4s5s
    *   雀頭: 2p2p (和了牌2p)
    *   待ち方: 2p単騎待ち (雀頭待ち)

2.  **符の計算:**
    *   **基本符:** 20符 (ピンフ形ではないため)
    *   **和了形による符:** 門前ロン (+10符)
    *   **面子構成による符:**
        *   東 (1z) の暗槓 (幺九牌): +32符
        *   紅中 (7z) の暗刻 (幺九牌): +8符
        *   2m3m4m の順子: 0符
        *   3s4s5s の順子: 0符
    *   **雀頭による符:** 2p (数牌の2-8) の雀頭: 0符
    *   **待ち形による符:** 2p 単騎待ち: +2符

    

In [19]:
df.to_csv(
    f"../dist/loop_multi_agents/evals-gen-question-with-loop-multi-agents-{datetime.now().strftime('%Y%m%d')}.csv",
    index=False,
)

In [20]:
import glob


def result_by_model(name: str):
    # 20250619のファイルを検索
    csv_files = glob.glob(f"../dist/{name}/*.csv")
    print(f"Found {len(csv_files)} CSV files:")
    for file in csv_files:
        print(f"  - {file}")

    # 各ファイルを読み込んでconcat
    dfs = []
    for file in csv_files:
        df = pd.read_csv(file)
        # ファイル名からモデル名を抽出
        model_name = file.split("-")[-2]  # 例: claude_sonnet
        df["model"] = model_name
        dfs.append(df)

    # 全てのデータを結合
    df = pd.concat(dfs, ignore_index=True)

    accuracy_by_model = (
        df.groupby("model")["correct"].agg(["count", "sum", "mean"]).round(3)
    )
    accuracy_by_model.columns = ["総問題数", "正解数", "正解率"]
    print("\n=== モデルごとの正解率 ===")
    print(accuracy_by_model)

    accuracy_by_model = (
        df.groupby("model")["is_error"].agg(["count", "sum", "mean"]).round(3)
    )
    accuracy_by_model.columns = ["総問題数", "エラー数", "エラー率"]
    print("\n=== モデルごとのエラー率 ===")
    print(accuracy_by_model)

In [21]:
result_by_model("zeroshot")

Found 3 CSV files:
  - ../dist/zeroshot/evals-gen-question-raw-prompt-gemini-20250629.csv
  - ../dist/zeroshot/evals-gen-question-raw-prompt-claude_sonnet-20250629.csv
  - ../dist/zeroshot/evals-gen-question-raw-prompt-deepseek-20250629.csv

=== モデルごとの正解率 ===
               総問題数  正解数   正解率
model                         
claude_sonnet    20    1  0.05
deepseek         20    0  0.00
gemini           20    6  0.30

=== モデルごとのエラー率 ===
               総問題数  エラー数  エラー率
model                          
claude_sonnet    20     0   0.0
deepseek         20     0   0.0
gemini           20     0   0.0


In [23]:
result_by_model("cot_and_rule")

Found 3 CSV files:
  - ../dist/cot_and_rule/evals-gen-question-with-cot-and-rule-prompt-deepseek-20250629.csv
  - ../dist/cot_and_rule/evals-gen-question-with-cot-and-rule-prompt-gemini-20250629.csv
  - ../dist/cot_and_rule/evals-gen-question-with-cot-and-rule-prompt-claude_sonnet-20250629.csv

=== モデルごとの正解率 ===
               総問題数  正解数  正解率
model                        
claude_sonnet    20    0  0.0
deepseek         20    0  0.0
gemini           20    6  0.3

=== モデルごとのエラー率 ===
               総問題数  エラー数  エラー率
model                          
claude_sonnet    20    18   0.9
deepseek         20     0   0.0
gemini           20     0   0.0


In [24]:
result_by_model("tools")

Found 3 CSV files:
  - ../dist/tools/evals-gen-question-with-tools-prompt-claude_sonnet-20250629.csv
  - ../dist/tools/evals-gen-question-with-tools-prompt-deepseek-20250629.csv
  - ../dist/tools/evals-gen-question-with-tools-prompt-gemini-20250629.csv

=== モデルごとの正解率 ===
               総問題数  正解数  正解率
model                        
claude_sonnet    20    2  0.1
deepseek         20    0  0.0
gemini           20    4  0.2

=== モデルごとのエラー率 ===
               総問題数  エラー数  エラー率
model                          
claude_sonnet    20    13  0.65
deepseek         20     0  0.00
gemini           20     0  0.00


In [ ]:
# result_by_model("sequencial_multi_agents")

Found 2 CSV files:
  - ../dist/multi_agents/evals-gen-question-with-sequential-agent-20250630.csv
  - ../dist/multi_agents/evals-gen-question-with-sequential-agent-20250706.csv

=== モデルごとの正解率 ===
       総問題数  正解数  正解率
model                
agent     6    3  0.5

=== モデルごとのエラー率 ===
       総問題数  エラー数   エラー率
model                   
agent     6     2  0.333


In [ ]:
result_by_model("loop_multi_agents")